In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch

In [3]:
table = pd.DataFrame({"outlook":['sunny','sunny','overcast','rain','rain','rain','overcast','sunny','sunny','rain','sunny','overcast','overcast','rain']
,"temperature":['hot','hot','hot','mild','cool','cool','cool','mild','cool','hot','mild','mild','hot','hot']
,"humidity":['hight','hight','hight','hight','normal','normal','normal','hight','normal','normal','normal','hight','normal','hight']
,"wind":['weak','strong','weak','weak','weak','strong','strong','weak','weak','weak','strong','strong','weak','strong']
,"played":['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no']})
table

,outlook,temperature,humidity,wind,played
0,sunny,hot,hight,weak,no
1,sunny,hot,hight,strong,no
2,overcast,hot,hight,weak,yes
3,rain,mild,hight,weak,yes
4,rain,cool,normal,weak,yes
5,rain,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,hight,weak,no
8,sunny,cool,normal,weak,yes
9,rain,hot,normal,weak,yes


### 自变量将决定因变量。其中，自变量是天气预报（outlook），温度（Temperature），湿度（Humidity）和风力（Wind），因变量是是否踢足球（Played football(yes/no)）。

In [4]:
data1 = pd.DataFrame({'天气':['晴','晴','阴','雨','雨','雨','阴','晴','晴','雨','晴','阴','阴','雨'],
                     '温度':['高','高','高','低','低','低','低','低','低','低','低','低','高','低'],
                     '湿度':['高','低','高','高','高','低','低','高','低','高','低','高','低','高'],
                     '起风':[False,True,False,False,False,True,True,False,False,False,True,True,False,True],
                     '打球':['NO','NO','YES','YES','YES','NO','YES','NO','YES','YES','YES','YES','YES','NO']})

def features_coder(set_):
    features_set = np.unique(set_)
    frequency = [(set_==element).sum() for element in features_set]
    return {element:[code,frequency[code]] for code,element in zip(range(features_set.shape[0]),features_set)}

def sorted_dict(dict_structure):
    return sorted(dict_structure.items(), key=lambda x: x[1], reverse=True)

def collaborative_filtering_jaccard(user,x,Lambda=1):
    scores = x.dot(np.ones_like(x[0]))*Lambda
    jaccard = np.array([jaccard_similarity(user,x[i]) for i in range(x.shape[0])])
    weighted_average = scores.dot(jaccard)/jaccard.dot(np.ones_like(jaccard))
    return weighted_average

def torch_cosine(vector1,vector2)->torch.Tensor:
    return vector1.mul(vector2).sum()/(vector1.norm()*vector2.norm())
#torch_cosine(tensorX[0],tensorX[0])

In [5]:
def coder_from_df(table):
    array = table.to_numpy()
    m ,n = array.shape
    result = []
    dictList = []
    for j in range(n):
        row = array[:,j]
        set_row = tuple(set(row))
        dict_row = {set_row[i]:i for i in range(len(set_row))}
        code_row = np.array([dict_row[element] for element in row])
        result.append(code_row)
        dictList.append(dict_row)
    return np.array(result).T,dictList

data , feature_dict = coder_from_df(table)

## 概率

In [6]:
data

array([[2, 0, 1, 1, 0],
       [2, 0, 1, 0, 0],
       [0, 0, 1, 1, 1],
       [1, 2, 1, 1, 1],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [2, 2, 1, 1, 0],
       [2, 1, 0, 1, 1],
       [1, 0, 0, 1, 1],
       [2, 2, 0, 0, 1],
       [0, 2, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0]])

In [7]:
#from typing_extensions import Literal

In [8]:
def prob(vector,element):
    n = vector.shape[0]
    return (vector==element).dot(np.ones_like(vector))/n

In [9]:
def prob_count(vector):
    distion = np.unique(vector)
    return {i:prob(vector,distion[i]) for i in range(distion.shape[0])}
prob_count(data[4])

{0: 0.2, 1: 0.8}

## 概率标量

In [10]:
def probability(info,element):
    return (info==element).dot(np.ones_like(info))

## 概率向量

In [11]:
def probability_vector(info):
    n = info.shape[0]
    distion = np.unique(info)
    if distion.shape[0]==1:
        return np.array([1])
    else:
        return np.array([probability(info,distion[i]) for i in range(distion.shape[0])])

In [12]:
probability_vector(data1[data1['打球']=='YES'][['打球']])

array([1])

## 熵向量

In [13]:
import math
def entropyPx(info,x,In='bit'):
    n = info.shape[0]
    px = np.dot(np.array(info==x).astype(int)/n,np.ones(n))
    if px == 0 :
        return px*0
    if px == 1:
        return 0
    else:
        if In == 'bit':
            return -px * np.log2(px)
        elif In == 'nat':
            return -px * math.log(px,math.e)
        
def entropy(X,In='bit'):
    H = np.array([entropyPx(X,x,In) for x in np.unique(X)])
    return H.dot(np.ones_like(H))


In [14]:
entropy(data1[data1['打球']=='YES']['打球']),entropy(data1['打球'])

(-3.2034265038149176e-16, 0.9402859586706311)

## 信息熵

## 条件熵

In [15]:
%time
def entropy_conditional(condition,info):
    n ,entropyVN = info.shape[0],[]
    bool_index = np.array([condition==e for e in np.unique(condition)])
    
    #TODO 熵
    for index_ in bool_index:
        info_cond = info[index_]
        entropy_cond = entropy(info_cond)
        entropyVN.append(entropy_cond)
    ent_cond = np.array(entropyVN)

    #TODO 概率
    mb,nb = bool_index.shape
    porb_cond = bool_index.dot(np.ones(nb))/nb
    
    return ent_cond.dot(porb_cond) 

entropy_conditional(data1["天气"],data1["打球"])

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


0.6935361388961918

## 信息增益

In [16]:
%time
def info_gain(condition,info):
    return entropy(info)-entropy_conditional(condition,info)

info_gain(data1["天气"],data1["打球"])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


0.24674981977443933

## 排序节点

In [17]:
def sorted_dict(dict_structure):
    return sorted(dict_structure.items(), key=lambda x: x[1], reverse=True)

def sort_node(condMat,yhat):
    return sorted_dict({row : info_gain(condMat[:,row],yhat) for row in range(condMat.shape[1])})

In [18]:

data = np.array(data1)
yhat = data[:,-1]
condMat = data[:,:-1]


In [42]:
def sort_gain(condMat,yhat):
    info_gain_dict , gains = dict(),[]
    for row in range(condMat.shape[1]):
        gain = info_gain(condMat[:,row],yhat)
        info_gain_dict.setdefault(row,gain)
    sorted_node = np.array(sorted_dict(info_gain_dict))
    check_zeros = sorted_node[sorted_node[:,1]>0]
    shape =  check_zeros.shape[0]
    if shape != 0:
        return check_zeros
    else:
        return None

def sort_node(condMat,yhat):
    check_zeros=sort_gain(condMat,yhat)
    if isinstance(check_zeros != None , bool) ==False:
        iG = check_zeros[:,0][0].astype(int)
        KG = check_zeros[:,0][1:].astype(int)
        node_data = condMat[:,iG]
        distion = np.unique(node_data)
        return distion,node_data,iG,check_zeros
    else:
        return np.array([None , None , None,None])

def tree(condMat,yhat,grains=None):

    distion,node_data,iG,check_zeros = sort_node(condMat,yhat)
    info_grains = [grains]
    info_grains.append([distion,node_data,check_zeros])
    
    if (distion != None).sum() == 3 :
        for e in distion:
            branch_node_index = node_data==e
            branch_node_condMat = condMat[branch_node_index]
            branch_node_clear = np.delete(branch_node_condMat,iG,axis=1)
            branch_node_yhat = yhat[branch_node_index]           
            return tree(branch_node_clear,branch_node_yhat,info_grains)
    else:
        return info_grains
        print(np.array(info_grains))

        
tree(condMat,yhat)

[[None,
  [array(['晴', '阴', '雨'], dtype=object),
   array(['晴', '晴', '阴', '雨', '雨', '雨', '阴', '晴', '晴', '雨', '晴', '阴', '阴',
          '雨'], dtype=object),
   array([[0.00000000e+00, 2.46749820e-01],
          [3.00000000e+00, 4.81270304e-02],
          [1.00000000e+00, 2.50781735e-02],
          [2.00000000e+00, 1.33974240e-03]])]],
 [array(['低', '高'], dtype=object),
  array(['高', '高', '低', '低', '低'], dtype=object),
  array([[0.        , 0.41997309],
         [1.        , 0.41997309],
         [2.        , 0.01997309]])]]

In [27]:
print(condMat.shape)
sort_node(condMat,yhat),condMat

(14, 4)


((array(['晴', '阴', '雨'], dtype=object),
  array(['晴', '晴', '阴', '雨', '雨', '雨', '阴', '晴', '晴', '雨', '晴', '阴', '阴',
         '雨'], dtype=object),
  0),
 array([['晴', '高', '高', False],
        ['晴', '高', '低', True],
        ['阴', '高', '高', False],
        ['雨', '低', '高', False],
        ['雨', '低', '高', False],
        ['雨', '低', '低', True],
        ['阴', '低', '低', True],
        ['晴', '低', '高', False],
        ['晴', '低', '低', False],
        ['雨', '低', '高', False],
        ['晴', '低', '低', True],
        ['阴', '低', '高', True],
        ['阴', '高', '低', False],
        ['雨', '低', '高', True]], dtype=object))

%time
sorted_node(data)

# pandas 计算

In [23]:
data1

,天气,温度,湿度,起风,打球
0,晴,高,高,False,NO
1,晴,高,低,True,NO
2,阴,高,高,False,YES
3,雨,低,高,False,YES
4,雨,低,高,False,YES
5,雨,低,低,True,NO
6,阴,低,低,True,YES
7,晴,低,高,False,NO
8,晴,低,低,False,YES
9,雨,低,高,False,YES


## 条件熵

In [24]:
%time
def ent(data:np.ndarray)->np.ndarray:
    prob1 = pd.value_counts(data) / len(data)
    import numpy as np
    return sum(np.log2(prob1)*prob1*(-1))
ent(data1['打球']),ent(data1['天气'])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


(0.9402859586706311, 1.5774062828523454)

## 计算条件信息熵

In [25]:
%time
#定义计算熵的函数
def ent(data):
    prob1 = pd.value_counts(data) / len(data)
    return sum(np.log2(prob1) * prob1 * (-1))
 
 
#定义计算信息增益的函数
def gain(data,str1,str2):
    e1 = data.groupby(str1).apply(lambda x:ent(x[str2]))
    p1 = pd.value_counts(data[str1]) / len(data[str1])
    e2 = sum(e1 * p1)
    return ent(data[str2]) - e2
 
 
#测试
gain(data1,'天气','打球')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


0.24674981977443933

## 计算信息增益